In [85]:
#Загрузим необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from icecream import ic
from sklearn.impute import SimpleImputer

# Условие

- Реализовать расчеты способами а и б для набора данных из файла insclass_train.csv/
2- Для 151 406 договоров страхования транспортных средств известны значения ряда признаков,
в том числе пол, возраст, стаж вождения и коэффициент бонус-малус водителя, тип, марка, модель,
год выпуска, страна – производитель, мощность и объем двигателя, а также признак target, равный 1,
 если заключение договора с клиентом является рисковым, и 0 в противном случае (файл insclass_train.csv).

- Требуется построить модель, предсказывающую значение признака target для 22 624 договоров из тестового набора данных (файл insclass_test.csv).

- В обучающем наборе данных для каждого договора известны следующие поля:

variable_1 - агрегированный коэффициент бонус-малус (повышающий или понижающий стоимость полиса в зависимости от аварийности в предыдущие периоды);

variable_2 - индикатор расторжения договора по инициативе страхователя (клиента);

variable_3 - индикатор расторжения договора по инициативе страховщика (страховой компании);

variable_4 - идентификатор года выпуска транспортного средства;

variable_5 - идентификатор страны - производителя транспортного средства;

variable_6 - мощность двигателя в лошадиных силах;

variable_7 - объем двигателя в куб. см;

variable_8 - идентификатор стороны расположения руля (левый или правый);

variable_9 - пробег транспортного средства, покрываемый гарантией производителя;

variable_10 - индикатор действия гарантии на транспортное средство;

variable_11 - "мультидрайв" - индикатор допуска к управлению транспортным средством более одного водителя;

variable_12 - возраст транспортного средства (в мес.);

variable_13 - возраст водителя с максимальным стажем;

variable_14 - коэффициент возраст-стаж;

variable_15 - коэффициент краткосрочности;

variable_16 - коэффициент мощности;

variable_17 - коэффициент "мультидрайв";

variable_18 - территориальный коэффициент;

variable_19 - коэффициент "КНДР";

variable_20 - идентификатор канала продаж;

variable_21 - марка транспортного средства;

variable_22 - модель транспортного средства;

variable_23 - индикатор отечественных транспортных средств;

variable_24 - пол водителя с максимальным коэффициентом "возраст-стаж";

variable_25 - индикатор пролонгации;

variable_26 - индикатор совпадения собственника транспортного средства и водителя;

variable_27 - стаж водителя с максимальным коэффициентом "возраст-стаж";

variable_28 - тип транспортного средства;

target - класс риска, равный 1, если заключение договора с клиентом является рисковым, и 0 в противном случае.

In [86]:
init_df = pd.read_csv('insclass_train.csv')
# считаем кол-во NaN
keys_with_nan = init_df.isna().sum()    
# ic(f'All NaN sum \n{keys_with_nan}')
keys_with_nan = keys_with_nan[keys_with_nan !=0]
keys_with_nan


variable_6        111
variable_7      95639
variable_8      29899
variable_9     134436
variable_12      1528
variable_13      1657
variable_14      2927
variable_15    151393
variable_16        12
variable_17        12
variable_18        12
variable_20         1
variable_24      2207
variable_27      2067
dtype: int64

In [87]:
refacted_df = init_df.copy().drop(['variable_15'], axis=1)
keys_with_nan = keys_with_nan.drop(labels=['variable_15'])
refacted_df.head()

,variable_1,variable_2,variable_3,variable_4,variable_5,variable_6,variable_7,variable_8,variable_9,variable_10,...,variable_20,variable_21,variable_22,variable_23,variable_24,variable_25,variable_26,variable_27,variable_28,target
0,w200,0,0,14,q2,98.0,NaN,0.0,NaN,0,...,C,j2,h45,0,0.0,0,1,19.323463,t1,0
1,w160,0,0,7,q11,106.0,NaN,0.0,NaN,0,...,C,j33,h234,0,1.0,0,1,41.177900,t1,0
2,w200,0,0,4,q3,123.0,NaN,0.0,NaN,0,...,B,j12,h28,0,0.0,0,1,3.614395,t1,0
3,w200,0,0,9,q3,102.0,NaN,0.0,NaN,0,...,C,j12,h64,0,1.0,0,0,49.041674,t1,0
4,w200,0,0,18,q20,117.0,NaN,0.0,NaN,0,...,C,j111,h991,0,1.0,0,0,17.909612,t1,0


In [88]:
nan_row_idx = refacted_df.loc[pd.isna(refacted_df['variable_20']), :].index[0]
ic(nan_row_idx)
ic(refacted_df.shape)
refacted_df = refacted_df.drop([nan_row_idx])
ic(refacted_df.shape)



ic| nan_row_idx: 109113
ic| refacted_df.shape: (151406, 28)


ic| refacted_df.shape: (151405, 28)


(151405, 28)

In [89]:
keys_with_nan = refacted_df.isna().sum()    
keys_with_nan = keys_with_nan[keys_with_nan !=0]
keys_with_nan

variable_6        111
variable_7      95639
variable_8      29898
variable_9     134435
variable_12      1528
variable_13      1657
variable_14      2927
variable_16        12
variable_17        12
variable_18        12
variable_24      2207
variable_27      2067
dtype: int64

In [90]:
refacted_df[keys_with_nan.index].info()

<class 'pandas.core.frame.DataFrame'>
Index: 151405 entries, 0 to 151405
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   variable_6   151294 non-null  float64
 1   variable_7   55766 non-null   float64
 2   variable_8   121507 non-null  float64
 3   variable_9   16970 non-null   float64
 4   variable_12  149877 non-null  float64
 5   variable_13  149748 non-null  float64
 6   variable_14  148478 non-null  float64
 7   variable_16  151393 non-null  float64
 8   variable_17  151393 non-null  float64
 9   variable_18  151393 non-null  float64
 10  variable_24  149198 non-null  float64
 11  variable_27  149338 non-null  float64
dtypes: float64(12)
memory usage: 15.0 MB


In [91]:

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(refacted_df[keys_with_nan.index])

refacted_df[keys_with_nan.index] = imputer.transform(refacted_df[keys_with_nan.index])
refacted_df.head()

,variable_1,variable_2,variable_3,variable_4,variable_5,variable_6,variable_7,variable_8,variable_9,variable_10,...,variable_20,variable_21,variable_22,variable_23,variable_24,variable_25,variable_26,variable_27,variable_28,target
0,w200,0,0,14,q2,98.0,1865.914572,0.0,3.459045e+32,0,...,C,j2,h45,0,0.0,0,1,19.323463,t1,0
1,w160,0,0,7,q11,106.0,1865.914572,0.0,3.459045e+32,0,...,C,j33,h234,0,1.0,0,1,41.177900,t1,0
2,w200,0,0,4,q3,123.0,1865.914572,0.0,3.459045e+32,0,...,B,j12,h28,0,0.0,0,1,3.614395,t1,0
3,w200,0,0,9,q3,102.0,1865.914572,0.0,3.459045e+32,0,...,C,j12,h64,0,1.0,0,0,49.041674,t1,0
4,w200,0,0,18,q20,117.0,1865.914572,0.0,3.459045e+32,0,...,C,j111,h991,0,1.0,0,0,17.909612,t1,0


In [92]:
keys_with_nan = refacted_df.isna().sum()    
keys_with_nan = keys_with_nan[keys_with_nan !=0]
keys_with_nan

Series([], dtype: int64)